### Topic modeling
Extracting topics from the reddit messages 

##### Imports 

In [19]:
# Imports
from bertopic import BERTopic
import pandas as pd
import numpy as np
import nltk 

##### Data 

Loading the data 

In [20]:
# getting subset of data 
messages_df = pd.read_csv("reddit_22_51/messages.csv", sep="\t")
messages_df.head()

id              user  \
0  j0s252k   HexagonOfVirtue   
1  j0s25h2           Teephex   
2  j0s25ht       1platesquat   
3  j0s25l5    YouLostTheGame   
4  j0s25nr  HMID_Delenda_Est   

                                                text  
0  im gonna find it  just to check, it's not the ...  
1  According to you criticizing and being skeptic...  
2  Gotcha. Can you explain to me why your opinion...  
3  Euros, which some argue is actually harder tha...  
4  You've been sounding more like PunishedSubSist...

Info on the data 

In [21]:
# info on data
column_list = messages_df.columns
shape = messages_df.shape

print("columns: ", column_list)
print("shape ", shape)

columns:  Index(['id', 'user', 'text'], dtype='object')
shape  (290898, 3)


**Data cleaning: stop word removal**

Stop word are very frequent words e.g. “the” and “a” that can impact the topics generated by the bertopic model due to their high frequency across most documents/texts - can be removed to get clearer more informative topics 

In [22]:
# METHOD 1: use a predefined list of stop words 
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stop_words(text, stop_words): 
    words = text.split() 
    # Use a list comprehension to remove stop words 
    filtered_words = [word for word in words if word.lower() not in stop_words] 
  
  # Join the filtered words back into a sentence 
    return ' '.join(filtered_words)

messages_df_sw = messages_df.copy()
messages_df_sw["text"] = messages_df_sw["text"].apply(lambda row: remove_stop_words(row, stop_words))
messages_df_sw

[nltk_data] Downloading package stopwords to /home/jnye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


id                  user  \
0       j0s252k       HexagonOfVirtue   
1       j0s25h2               Teephex   
2       j0s25ht           1platesquat   
3       j0s25l5        YouLostTheGame   
4       j0s25nr      HMID_Delenda_Est   
...         ...                   ...   
290893  j1nzngu   Electrical-Wish-519   
290894  j1nznu5         peacefulshit9   
290895  j1nzo0f              Realeron   
290896  j1nzo1x      DosTristesTigres   
290897  j1nzo39  Temporary_Scene_8241   

                                                     text  
0       im gonna find check, ai art one right? trigger...  
1       According criticizing skeptical active politic...  
2                   Gotcha. explain opinion important me?  
3                         Euros, argue actually harder WC  
4                     sounding like PunishedSubSister tbh  
...                                                   ...  
290893  Yeah. way last sentence comes off, took like “...  
290894  Sorry. English sucks ! must good foreign langu...  
290895                    Ouch! Black Panthers precedent.  
290896                                 Ghost totally move  
290897  Definitely. established diligently. trying swa...  

[290898 rows x 3 columns]

In [23]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [24]:
# sklearn method
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Print the full list of stopwords
print(sorted(ENGLISH_STOP_WORDS))
print(len(ENGLISH_STOP_WORDS))

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give

In [25]:
# METHOD 2: remove most frequent words found in the data 
most_frequent_words = pd.Series(' '.join(messages_df['text']).lower().split()).value_counts()[:100].index.tolist()

messages_df_sw2 = messages_df.copy()
messages_df_sw2["text"] = messages_df_sw2["text"].apply(lambda row: remove_stop_words(row, most_frequent_words))

Convert data to list of strings (input needed by bertopic)

In [26]:
# convert to list of strings (input needed by bertopic model)
messages_list = messages_df_sw["text"].astype(str).tolist()
len(messages_list)

290898

Get subset of data to work with 

In [27]:
# subset of data - 5000 rows
messages_subset = messages_list[:10000]
messages_subset

['im gonna find check, ai art one right? triggered rare',
 'According criticizing skeptical active politican mental disorder?',
 'Gotcha. explain opinion important me?',
 'Euros, argue actually harder WC',
 'sounding like PunishedSubSister tbh',
 'hate people race hate people born',
 'communism abysmal failure ever single instances attempted? Yeah. Hans would agree.',
 'made genuine effort try estimate cost moving affordable place?',
 'implemented rule saying link social media sites (Instagram, facebook, Mastodon, etc)',
 'Yes. Every single human earth eats tiny amounts poison regular basis. know know that? literally everything poisonous depending dose. Water poisonous drink enough it.',
 'set base couple days. Go read Volume 2 AFH 10-222 see done.',
 'welcome opinion.',
 'Literally least worst Trump adjacent person could sell to.',
 'Speech thought. Thought police.',
 'know happened school system.',
 'Thread OP: "so need landlords?" Me: "unironically yes" tbh long one rental company g

##### Bertopic model 

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english")

# topic_model = BERTopic()
topic_model = BERTopic(vectorizer_model=CountVectorizer(stop_words="english")) 
# topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [29]:
# fitting the bertopic model 
topic_model_fitted = topic_model.fit(messages_subset)

In [30]:
# parameters of the general model
topic_model_fitted.get_params()

{'calculate_probabilities': False,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': <bertopic.backend._sentencetransformers.SentenceTransformerBackend at 0x7fc9b2b8e890>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 1),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, low_memory=False, metric='cosine', min_dist=0.0, n_components=5, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(stop_words='english'),
 'verbose': False,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [31]:
# get info on topics (names, important words, representative reddit message/document)
topic_info = topic_model_fitted.get_topic_info()
topic_info.to_csv("topic_info_10000_method1.csv", sep="\t", index=False)

In [32]:
topic_info

Topic  Count                                   Name  \
0       -1   4486                -1_like_im_people_think   
1        0    325      0_congress_court_constitution_fbi   
2        1    212              1_housing_roads_rent_city   
3        2    182  2_capitalism_socialism_communism_marx   
4        3    137     3_ukraine_russia_russian_ukrainian   
..     ...    ...                                    ...   
142    141     11  141_illusion_philosophy_negated_stiff   
143    142     11           142_dt_libtard_silly_artemis   
144    143     11             143_games_tv_directtv_slot   
145    144     10         144_algebra_progress_lbs_learn   
146    145     10      145_subreddit_post_separate_fluff   

                                        Representation  \
0    [like, im, people, think, right, time, thats, ...   
1    [congress, court, constitution, fbi, doj, trum...   
2    [housing, roads, rent, city, cities, property,...   
3    [capitalism, socialism, communism, marx, capit...   
4    [ukraine, russia, russian, ukrainian, nato, wa...   
..                                                 ...   
142  [illusion, philosophy, negated, stiff, boils, ...   
143  [dt, libtard, silly, artemis, magma, boredom, ...   
144  [games, tv, directtv, slot, salt, lake, month,...   
145  [algebra, progress, lbs, learn, arrogant, humb...   
146  [subreddit, post, separate, fluff, 5050, typin...   

                                   Representative_Docs  
0    [little hypocritical knowing Elon officially b...  
1    [actually read dred Scott case. case: case ini...  
2    [unless LARGE group left isolated large group ...  
3    [Look, get post vague attempt (I don’t think d...  
4    [appreciate these: cyber cat-and-mouse game en...  
..                                                 ...  
142  [Oh yeah, read philosophy scrutiny particularl...  
143  [u/MasterOfLords1 literally post DT instead go...  
144  [Yeah would insane deal. Right NFL Sunday Tick...  
145  [Strategy? wrong you? strategy. think twenties...  
146  [imo get fuck subreddit, post felt like lot fl...  

[147 rows x 5 columns]

In [33]:
# get the top words for each topic id (form the topic names)
topic_model_fitted.get_topics()

{-1: [('like', np.float64(0.005457766756503826)),
  ('im', np.float64(0.005241239288523216)),
  ('people', np.float64(0.005198990631854042)),
  ('think', np.float64(0.00497459671874603)),
  ('right', np.float64(0.004489606823968064)),
  ('time', np.float64(0.004434245286741447)),
  ('thats', np.float64(0.004235269518283284)),
  ('want', np.float64(0.00418942080252323)),
  ('say', np.float64(0.004176479405618831)),
  ('trump', np.float64(0.004133370648587436))],
 0: [('congress', np.float64(0.019824662583576104)),
  ('court', np.float64(0.01752158949664019)),
  ('constitution', np.float64(0.015682087287888634)),
  ('fbi', np.float64(0.015109914746788855)),
  ('doj', np.float64(0.013548522762800224)),
  ('trump', np.float64(0.012798855784095134)),
  ('law', np.float64(0.012492197507038914)),
  ('arms', np.float64(0.011484721234610456)),
  ('charges', np.float64(0.011005167451801825)),
  ('justice', np.float64(0.010344081427403054))],
 1: [('housing', np.float64(0.028965133719964332)),
  